
# Digit Recognition


## Introduction

This notebook is concerned with explaining the Digit recognition script that I wrote in relation to the MNIST dataset.
In the previous notebook, I went through how to read in the MNIST dataset and also how to save the images to the comoputers file system. I have re used some of that code in this script, so this notebook will mainly be in relation to the parts of the script that have nothing to do with that.

Artificial neural networks are essentially computing systems that were designed based on the biological neural networks found in animals. If you wish to know more about the theory behind neural networks please visit [wikipedia](https://en.wikipedia.org/wiki/Artificial_neural_network).

Each neural network has at least one input layer, one hidden layer and one output layer. Data comes in through the input layer, then an activation fucnction is executed on the data in the the hidden layer/s, and finally the data is is then output. The particular kind of neural network we will be working with is called a "fully connected", or "dense" network, because each node of each layer is connected to every node in the next layer. This will make more sense when I start to go through the program.

![](img/neural.jpeg)


## Imports

In order to be able to use the tensorflow and keras imports, I needed to set up a tensorflow environment using anaconda. For more information on this please visit [Tensor flow environment set up](https://towardsdatascience.com/setup-an-environment-for-machine-learning-and-deep-learning-with-anaconda-in-windows-5d7134a3db10).

In [1]:
import gzip ### For reading in the gzip files.
import numpy as np ### For work with arrays.
import tensorflow as tf ### For using keras.


In [2]:
import numpy as np
import keras as kr ### For building and operating the model.
import sklearn.preprocessing as pre ### For training.
import sys ### For IO ans system exit(0).
import cv2 ### For reading and writing images.
from PIL import Image ### For manipulating images()

Using TensorFlow backend.
